In [1]:
# Import the required libraries
from selenium import webdriver
import pandas as pd
from tqdm import tqdm
import numpy as np
import os

In [2]:
# Start the chrome driver, configuring this on the local requires
# some time. See this for installation
# Note: this would run chromium not chrome although simply find command to install and run chrome
# https://stackoverflow.com/questions/22476112/using-chromedriver-with-selenium-python-ubuntu
driver = webdriver.Chrome()

# Selenium and driver
Using selenium in chrome driver simply means we are going to run the chrome as a user using selenium. So we have functions to click on what we want and go through the browser as a user would. We have the added benefit that we can extract the tables in the webpage using selenium as well. This would be used in out task here.

## Steps:
1. Get to the main page using link and issueing command driver.get()
2. Fill the form. To fill the form you have to click on the radio buttons. We have to this activity through selenium as well. First we have to get that radio button we want to click upon as a selenium element. Use the complete xpath to locate the html element you want to click. To get the complete xpath of any element in the browser follow [this link](https://www.google.com/search?q=how+to+get+complete+xpath+in+chrome&oq=get+complete+xpath+&aqs=chrome.1.69i57j0l2.5527j0j4&sourceid=chrome&ie=UTF-8#kpvalbx=_tauiXvbSGZ6a4-EPwa-FmAc42). Once you have the xpath use the driver.find_element_by_xpath() function which returns the required html element. Once you have the selenium html element, do .click() to click on the radio button.
3. Once form is filled by clicking on required radio buttons, use the same technique to get the selenium element of submit button. Then do .click to open the search results.
4. Search results generally comprise of different pages, we are going to use the following process to get each page.
5. We will extract the entire page table, then we will click on the next page using technique discussed above.
6. We iterate this process and append the array in one dataframe, till all the pages are extracted.
7. Then we get the data in csv using .to_csv() method in pandas dataframe.

In [3]:
# To get the main page
driver.get('https://pfaf.org/user/Default.aspx')

In [4]:
# Get the selenium element to click on the salinity radio button
salinity_element = driver.find_element_by_xpath("/html/body/form/div[9]/div[2]/table/tbody/tr[52]/td[2]/div/table/tbody/tr/td[6]/input")

In [5]:
# Click on the radio button to select the salinity
salinity_element.click()

In [6]:
# Get the submit html element and click
submit_button = driver.find_element_by_xpath("/html/body/form/div[9]/div[3]/input[1]")
submit_button.click()

In [7]:
# Function to extract one table fromt the browser
def scrap(driver):
    table = []
    names = driver.find_elements_by_xpath('//tr[@class="content"]')
    for i in tqdm(range(len(names))):
        if i!=0:
            a = names[i].find_elements_by_tag_name('td')
        else:
            a = names[i].find_elements_by_tag_name('th')
        res = []
        for i in a:
            res.append(i.text)
        table.append(res)
    return table

In [8]:
# Get the tables from each page
res = []
for i in range(4):
    table = scrap(driver)
    df_table = pd.DataFrame(table[1:], columns=table[0])
    res.append(df_table)
    
    if i!=3:
        next_page = driver.find_element_by_xpath("/html/body/form/div[7]/table/tbody/tr[2]/td/div[3]/table/tbody/tr[102]/td/table/tbody/tr/td["+str(i+2)+"]")
        next_page.click()

100%|██████████| 101/101 [00:10<00:00,  9.38it/s]


In [9]:
res = pd.concat(res)

In [10]:
res

,LatinName,CommonName,pH,EdibilityRating,MedicinalRating,OtherUsesRating
0,Acacia auriculiformis,"Ear-Pod Wattle, Black Acacia, Earleaf, Black w...",AN,1,0,4
1,Acacia farnesiana,"Sweet Acacia, Perfume Acacia, Huisache",ANB,2,2,
2,Acacia longifolia,"Sydney Golden Wattle, Acacia",ANB,3,0,
3,Acacia mucronata,Narrow-Leaf Wattle,ANB,2,0,
4,Acacia retinodes,"Swamp Wattle, Water wattle",ANB,2,0,
...,...,...,...,...,...,...
95,Caesalpinia echinata,"Pau Brasil, Brazil Wood, Indian Savin",NB,0,2,4
96,Calandrinia balonensis,,ANB,2,0,
97,Calandrinia polyandra,Parakeelya,ANB,2,0,
98,Calandrinia remota,,ANB,2,0,


In [11]:
# Save the dataframe as csv
res.to_csv("salinity.csv")

In [12]:
# close the driver
driver.close()